In [57]:
# import packages
import pandas as pd
from gurobipy import *

# read excel sheets with pandas
data = pd.ExcelFile('SuperChipData.xlsx')
productionCapacityDf = pd.read_excel(data, 'Production Capacity')
salesRegionDemandDf = pd.read_excel(data, 'Sales Region Demand')
shippingCostsDf = pd.read_excel(data, 'Shipping Costs')
productionCostsDf = pd.read_excel(data, 'Production Costs')

# preparing for adding variables
# total number of sales region j
salesRegion = 23
# total number of computer chip z
computerChip = 30
# [sales region, computer chip, yearly demand(thousands)]
salesRegionDemand = salesRegionDemandDf.to_numpy()
# [facility name, facility yearly production capacity(thousands)]
productionCapacity = productionCapacityDf.to_numpy()
# [facility, computer chip, production cost per chip ($)]
productionCosts = productionCostsDf.to_numpy()
# [facility, computer chip, sales region, shipping cost per chip($)]
shippingCosts = shippingCostsDf.to_numpy()

# initialize model
m = Model()
# set objective function to be minimization
m.modelSense = GRB.MINIMIZE

# declare variables and add them to model
# facility set S
facilitySet = ["Alexandria", "Richmond", "Norfolk", "Roanoke", "Charolottesville"]
# assign number to facility
facilityDict = {}
for i, f in enumerate(facilitySet):
     facilityDict[f] = i + 1

# Djz = z chip yearly demand in sales region j (thousands unit)
D = {}
for d in salesRegionDemand:
        # (j, z) = demand in j z
        D[d[0], d[1]] = d[2]

# Ci = yearly chip production capacity in facility i (thousands unit)
C = {}
for c in productionCapacity:
     # i = production capacity in i
     C[facilityDict[c[0]]] = c[1]

# Piz = production cost per z chip in facility i ($)
P = {}
for p in productionCosts:
     # (i, z) = production cost in i, z
     P[facilityDict[p[0]], p[1]] = p[2]

# Sijz = shipping cost per z chip from facility i to sales region j ($)
S = {}
for s in shippingCosts:
     # (i, j, z) = shipping cost per z from i to j
     S[facilityDict[s[0]], s[2], s[1]] = s[3]

# Xijz = amount of z chips shipped from facility i to sales region j (unit)
X = {}
# iterate through facility set, sales region numbers and computer chip numbers
for i in facilitySet:
    for j in range(1, salesRegion + 1):
        for z in range(1, computerChip + 1):
            X[facilityDict[i], j, z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                                name = f"X_{i}_{j}_{z}")

# Yiz = amount of z chips produced in facility i (unit)
Y = {}
# iterate through facility set, computer chip numbers
for i in facilitySet:
    for z in range(1, computerChip + 1):
        Y[facilityDict[i], z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                         name = f"Y_{i}_{z}")
        
# notify model the changes
m.update()

# set the objective function
m.setObjective(quicksum(S[facilityDict[i], j, z] * X[facilityDict[i], j, z] for i 
                        in facilitySet for j in range(1, salesRegion + 1) for z in 
                        range(1, computerChip + 1)) + 
               quicksum(P[facilityDict[i], z] * Y[facilityDict[i], z] for i 
                        in facilitySet for z in range(1, computerChip + 1)))

# first constraint ∑Yiz <= 1000Ci
facilityProductionConst = {}
for i in facilitySet:
     facilityProductionConst[i] = m.addConstr(quicksum(
          Y[facilityDict[i], z] for z in range(1, computerChip + 1)
          ) <= 1000 * C[facilityDict[i]], name = f"production_{i}")

# second constraint ∑Xijz = 1000Djz
shipDemandConst = {}
for j in range(1, salesRegion + 1):
     for z in range(1, computerChip + 1):
          shipDemandConst[j, z] = m.addConstr(quicksum(
               X[facilityDict[i], j, z] for i in facilitySet) == 1000 * D[j, z],
               name = f"shipping_demand_{j}_{z}")

# third constraint Yiz >= ∑Xijz
productionShipConst = {}
for i in facilitySet:
     for z in range(1, computerChip + 1):
          productionShipConst[facilityDict[i], z] = m.addConstr(
               Y[facilityDict[i], z] >= quicksum(
                    X[facilityDict[i], j, z] for j in range(1, salesRegion + 1)), 
                    name = f"production_shipping_{i}_{z}")

# for the fourth and fifth constraints, since Xijz and Yiz both have lower bound of
# 0 on initialization, not adding them here

# notify model the changes
m.update()

# trigger optimization
m.optimize()

# print the object function value
print ("Optimal Production and Distribution Cost = ", m.objVal)
# write model to file
m.write("Computer-Chip-Project-Model.lp")
# write solution to file
m.write("Computer-Chip-Project-Model.sol")



Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 845 rows, 3600 columns and 7200 nonzeros
Model fingerprint: 0x756da770
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+05]
Presolve removed 151 rows and 155 columns
Presolve time: 0.00s
Presolved: 694 rows, 3445 columns, 6890 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9059636e+07   7.737500e+03   0.000000e+00      0s
      45    4.9083430e+07   0.000000e+00   0.000000e+00      0s

Solved in 45 iterations and 0.01 seconds (0.01 work units)
Optimal objective  4.908343040e+07
Optimal Production and Distribution Cost =  49083430.39999999
